In [1]:
import numpy as np
import pandas as pd
import urllib

In [2]:
# TITLE = 'truck driver'
# TITLE = 'registered nurse'
# TITLE = 'police officer'
# TITLE = 'real estate agent'
# TITLE = 'phlebotomist'
TITLE = 'software engineer'

df_vol = pd.read_csv('../input/volume_{}.csv'.format(TITLE.replace(" ", "_")))
df_paa = pd.read_csv('../output/paa_{}.csv'.format(TITLE.replace(" ", "_")))
df_vol.columns = ['query', 'volume']
df_paa.columns = ['query', 'rank', 'question', 'answer', 'link']
print(df_paa.shape)
df_paa = df_paa.dropna(subset=['query', 'question', 'answer'])
print(df_paa.shape)
df_paa = df_paa[df_paa.apply(lambda x: "{" not in x['query'], axis=1)]
print(df_paa.shape)
df_paa.head()

(3491, 5)
(3491, 5)
(3491, 5)


,query,rank,question,answer,link
0,software engineer salaries,0,How much can I earn as a software engineer?,Software engineering is a smart career choice ...,https://learning.linkedin.com/blog/tech-tips/t...
1,software engineer salaries,1,How much do software engineers earn UK?,The average Software Engineer can expect to ma...,https://jobs.telegraph.co.uk/article/software-...
2,software engineer salaries,2,What is the average salary in Kuwait for softw...,A person working as a Software Engineer in Kuw...,http://www.salaryexplorer.com/salary-survey.ph...
3,software engineer salaries,3,Is software engineer a good career?,You got: Software Developer Isn't a Good Caree...,https://www.thebalancecareers.com/should-you-b...
4,software engineer salaries,4,How much do software engineers earn UK?,The average Software Engineer can expect to ma...,https://jobs.telegraph.co.uk/article/software-...


In [3]:
df_vol['volume'] = (df_vol['volume']-np.min(df_vol['volume']))/(np.max(df_vol['volume'])-np.min(df_vol['volume']))
df_paa['rank'] = (np.max(df_paa['rank'])-df_paa['rank'])/(np.max(df_paa['rank'])-np.min(df_paa['rank']))+0.1
df_merged = df_paa.merge(df_vol, how='left', on='query')
default_vol = np.mean(df_vol['volume'])
print(default_vol)
df_merged = df_merged.fillna(default_vol)
# df_merged['score'] = df_merged['rank']*0.5+df_merged['volume']*0.5
df_merged['score'] = df_merged['rank']*df_merged['volume']

df_group = df_merged.groupby(['question', 'link']).agg({'answer':set, 'query':set, 'score':sum, 'rank':"count"}).reset_index()
df_group = df_group.rename(columns={'rank':'cnt'})
df_group = df_group.sort_values(by='score', ascending=False).reset_index(drop=True)
print(df_group.shape[0])
print(sum(df_group.apply(lambda x: len(x['answer']), axis=1)))

0.019611735609651192
930
939


In [4]:
pd.set_option('display.max_colwidth', 200)
df_group.head()

,question,link,answer,query,score,cnt
0,How much do software engineers earn UK?,https://jobs.telegraph.co.uk/article/software-engineer-salary/,"{The average Software Engineer can expect to make 38,000 per year, while the average salary for senior level employees ranges from 50,000 to 79,000 per annum, this reveals the value of experience ...","{software engineer annual wage, wage as software engineer, software engineer income, software engineer yearly income, software engineer annual salary, average software engineer wage, how much does...",2.040004,25
1,What is the highest paid software engineer?,https://devskiller.com/software-engineer-jobs-pay-the-most/,"{3. The highest paying software engineer jobs cluster in certain places $187,236 Cupertino. $169,637 Mountain View. $168,974 Menlo Park. $151,908 San Francisco. $150,948 San Jose. $147,201 R...","{whats the average pay for a software engineer, software engineer salary, average software engineer salary per month, software engineer income, software engineer salary per month, software enginee...",1.747212,24
2,How much can I earn as a software engineer?,https://learning.linkedin.com/blog/tech-tips/the-american-city-that-pays-software-engineers-the-most--and-oth,"{Software engineering is a smart career choice it's one of the most in-demand skills in the world and American software engineers earn a median salary of $112,000. But all software engineers aren...","{software engineer annual wage, software engineer yearly wage, software engineer annual income, software engineer pay per hour, software engineer salaries, wage as software engineer, software engi...",1.741488,14
3,What is the highest salary of a software engineer?,https://www.quora.com/What-is-the-highest-salary-a-software-engineer-can-get,{But the highest salaries that I've found are: Dropbox - $291K. Lyft - $291K. Netflix - $288K. Facebook - $271K. Airbnb - $271K. Uber - $262K.},"{average software engineer salary, average pay for software engineer, software engineer yearly pay, how much software engineer get paid an hour, software engineer annual salary, average software e...",1.729339,133
4,What is a good software engineer starting salary?,https://www.codingdojo.com/blog/entry-level-software-developer-salary,"{According to GlassDoor.com, which gathered information from over 15,000 software developers, the national average entry level software developer salary is $55,000. That comes to approximately $26...","{software engineer annual wage, whats the average pay for a software engineer, average software engineer salary, wage as software engineer, average pay for software engineer, software engineer yea...",1.305892,58


In [6]:
df_qna = df_group.copy()
df_qna['rank'] = df_qna.index
df_qna['topic'] = ""
df_qna['answer'] = df_qna.apply(lambda x: list(x['answer'])[0]+" "+str(x['link']), axis=1)
df_qna = df_qna[['topic', 'question', 'rank', 'answer']]
df_qna.head()

,topic,question,rank,answer
0,,How much do software engineers earn UK?,0,"The average Software Engineer can expect to make 38,000 per year, while the average salary for senior level employees ranges from 50,000 to 79,000 per annum, this reveals the value of experience w..."
1,,What is the highest paid software engineer?,1,"3. The highest paying software engineer jobs cluster in certain places $187,236 Cupertino. $169,637 Mountain View. $168,974 Menlo Park. $151,908 San Francisco. $150,948 San Jose. $147,201 Re..."
2,,How much can I earn as a software engineer?,2,"Software engineering is a smart career choice it's one of the most in-demand skills in the world and American software engineers earn a median salary of $112,000. But all software engineers aren'..."
3,,What is the highest salary of a software engineer?,3,But the highest salaries that I've found are: Dropbox - $291K. Lyft - $291K. Netflix - $288K. Facebook - $271K. Airbnb - $271K. Uber - $262K. https://www.quora.com/What-is-the-highest-salary-a-sof...
4,,What is a good software engineer starting salary?,4,"According to GlassDoor.com, which gathered information from over 15,000 software developers, the national average entry level software developer salary is $55,000. That comes to approximately $26...."


In [7]:
file_qna = '../output/qna_{}.csv'.format(urllib.parse.quote_plus(TITLE))
df_qna.to_csv(file_qna, index=False)